# Evolutionary transition rates of leaf endopolyploidy

James Seery (jseery@mail.uoguelph.ca)

## Setting up

#### Load packages and functions

In [58]:
tryCatch(    # Load corHMM package. Provides the functions for ancestral reconstruction and inference of transition rates.
    library(corHMM),
    error = install.packages("corHMM", repos = "http://cran.utstat.utoronto.ca/")
)
tryCatch(    # Load CAPER package. Provides the ability to prune a large phylogeny.
    library(caper),
    error = install.packages("caper", repos = "http://cran.utstat.utoronto.ca/")
)
source("corHMM_functions.R")

Installing package into 'C:/Users/CCCP/Documents/R/win-library/3.2'
(as 'lib' is unspecified)
Warning message:
: package 'corHMM' is in use and will not be installedInstalling package into 'C:/Users/CCCP/Documents/R/win-library/3.2'
(as 'lib' is unspecified)
Warning message:
: package 'caper' is in use and will not be installedWarning message:
In file(filename, "r", encoding = encoding): cannot open file 'corHMM_functions.R': No such file or directory

ERROR: Error in file(filename, "r", encoding = encoding): cannot open the connection


#### Import data

##### Figure out what is the best indicator of endopolyploidy

In [170]:
setwd("../../Raw_data")
endo.data = read.csv("Endo_literature_data/Mined+summer_full.csv")

In [37]:
print("How many entries for leaf endoreduplication index (EI)?")
length(endo.data$EI[!is.na(endo.data$EI)])
print("How many entries for mean leaf ploidy?")
length(endo.data$Mean.ploidy[!is.na(endo.data$Mean.ploidy)])

[1] "How many entries for leaf endoreduplication index (EI)?"


[1] 243

[1] "How many entries for mean leaf ploidy?"


[1] 186

##### Make subset that only has EI.

In [181]:
endo = endo.data[, c(2,4,6)]
names(endo) = c("name", "state", "state2")
endo = subset(endo, !is.na(endo$state))

In [182]:
summary(endo$state)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
0.00000 0.02230 0.04516 0.17670 0.14740 2.31800 

##### Turn endopolyploidy into a character with two states

In [183]:
threshold = 0.1 # Specify threshold endoreduplication index for deciding whether a species has endopolyploid leaves.
state = numeric(length(endo$state)) + 0
state[endo$state < threshold] = 1
state[endo$state >= threshold] = 2
print("Proportion of species in state 2:")
length(state[state == 2]) / length(state)
endo$state = state

[1] "Proportion of species in state 2:"


[1] 0.3127572

##### Turn endopolyploidy and woodiness into a three-state character

In [184]:
print("These are the species that are woody endopolyploids")
endo.data$Species[endo.data$EI > 0.1 & endo.data$GH == "W"]

[1] "These are the species that are woody endopolyploids"


[1] Opuntia_tomentosa Sophora_japonica 
248 Levels: Acer_nigrum Acer_saccharinum Acer_saccharum ... Zea_mays

For now we'll treat woody endopolyploid species as non-woody

In [190]:
state2 = numeric(length(endo$state)) + 0
state2[endo$state2 == "W"] = 1
state2[endo$state2 == "H"] = 2
state2[endo$state == 2] = 3 # Represents non-woody endopolyploidy
endo$state2 = state2

#### Import entire Zanne phylogeny (phylo)

In [60]:
setwd("../../Raw_data")
phylo = read.tree("Zanne_phylo/Zanne.nwk")
phylo = makeLabel(phylo) # Add unique labels to all unnamed nodes

#### Match the phylogeny (~30,000 species) to the dataset (~168 species)

In [191]:
data = comparative.data(phy = phylo,
                        data = endo,
                        names.col = "name")

##### Extract matched species

In [193]:
endo.matched = data.frame(data$phy$tip.label, data$data)
names(endo.matched) = c("V1", "V2", "V3")
#endo.matched$V2 = as.factor(endo.matched$V2)
rownames(endo.matched) = NULL

## Modelling evolutionary transition rates of leaf endopolyploidy

#### Symmetric transition model

In [194]:
Sym_model = rayDISC(phy = data[[1]],
                    data = endo.matched,
                    model = "ER",
                    node.state = "marginal"
                   )

Sym_model

State distribution in data:
States:	1	2	
Counts:	145	61	
Initializing... 
Finished. Beginning thorough search... 
Finished. Inferring ancestral states using marginal reconstruction. 
Finished. Performing diagnostic tests. 



Fit
      -lnL     AIC     AICc ntax
 -100.1455 202.291 202.3106  206

Rates
            1           2
1          NA 0.005577381
2 0.005577381          NA

Arrived at a reliable solution 

#### Asymmetric transition models

In [165]:
Asym_model = rayDISC(phy = data[[1]],
                    data = endo.matched,
                    model = "ARD",
                    node.state = "marginal"
                   )

Asym_model

State distribution in data:
States:	1	2	
Counts:	145	61	
Initializing... 
Finished. Beginning thorough search... 
Finished. Inferring ancestral states using marginal reconstruction. 
Finished. Performing diagnostic tests. 



Fit
      -lnL     AIC     AICc ntax
 -93.15551 190.311 190.3701  206

Rates
           1           2
1         NA 0.005610422
2 0.01798226          NA

Arrived at a reliable solution 

##### Compare models using a $\chi^2$ test of ln(likelihood) scores

In [168]:
comp.Asym.Sym = 2*(Asym_model$loglik - Sym_model$loglik) ###order of function should have the better fitting model (higher loglik) listed first
comp.Asym.Sym
pchisq(comp.Asym.Sym, df = 1, lower.tail=FALSE)

[1] 13.98001

[1] 0.0001847643

#### Asymmetric model is significantly better

## Modelling transitions of endopolyploidy and woodiness

We have at least one species that is woody and endopolyploidy. For now it has been binned into non-woody endopolyploid species.

In [195]:
Sym_wood_model = rayDISC(phy = data[[1]],
                        data = endo.matched,
                        charnum = 2, # Specifies the endo+woody trait
                        model = "ER",
                        node.state = "marginal"
                       )

Sym_wood_model

State distribution in data:
States:	1	2	3	
Counts:	52	93	61	
Initializing... 
Finished. Beginning thorough search... 
Finished. Inferring ancestral states using marginal reconstruction. 
Finished. Performing diagnostic tests. 



Fit
      -lnL     AIC     AICc ntax
 -143.1275 288.255 288.2746  206

Rates
            1           2           3
1          NA 0.004194609 0.004194609
2 0.004194609          NA 0.004194609
3 0.004194609 0.004194609          NA

Arrived at a reliable solution 

In [206]:
Asym_wood_model = rayDISC(phy = data[[1]],
                        data = endo.matched,
                        charnum = 2, # Specifies the endo+woody trait
                        model = "ARD",
                        node.state = "marginal"
                       )

Asym_wood_model

State distribution in data:
States:	1	2	3	
Counts:	52	93	61	
Initializing... 
Finished. Beginning thorough search... 
Finished. Inferring ancestral states using marginal reconstruction. 
Finished. Performing diagnostic tests. 



Fit
      -lnL      AIC     AICc ntax
 -127.8692 267.7384 268.1605  206

Rates
             1           2           3
1           NA 0.003235799 0.004727352
2 0.0006196325          NA 0.008136740
3 0.0000000000 0.018835981          NA

Arrived at a reliable solution 

##### Compare models using a $\chi^2$ test of ln(likelihood) scores

In [197]:
comp.Asym.Sym_wood = 2*(Asym_wood_model$loglik - Sym_wood_model$loglik) ###order of function should have the better fitting model (higher loglik) listed first
comp.Asym.Sym_wood
pchisq(comp.Asym.Sym_wood, df = 1, lower.tail=FALSE)

[1] 30.51668

[1] 3.310084e-08

### See if better fit can be obtained by making rates 4 and 6 equal

In [210]:
trans_mat_eq46 = rate.mat.maker(hrm = FALSE, ntraits = 1, nstates = 3, model = "ARD")
trans_mat_eq46 = rate.par.eq(trans_mat_eq46, eq.par = c(4, 6))

In [211]:
Asym_wood_model_eq46 = rayDISC(phy = data[[1]],
                        data = endo.matched,
                        charnum = 2, # Specifies the endo+woody trait
                        model = "ARD",
                        node.state = "marginal",
                        rate.mat = trans_mat_eq46
                       )

Asym_wood_model_eq46

State distribution in data:
States:	1	2	3	
Counts:	52	93	61	
Initializing... 
Finished. Beginning thorough search... 
Finished. Inferring ancestral states using marginal reconstruction. 
Finished. Performing diagnostic tests. 



Fit
      -lnL      AIC     AICc ntax
 -130.2224 270.4448 270.7448  206

Rates
             1           2            3
1           NA 0.007687841 0.0003588803
2 0.0005671618          NA 0.0123882093
3 0.0000000000 0.012388209           NA

Arrived at a reliable solution 

#### Is the totally asymmetric model still better?

In [212]:
comp.Asym_wood_eq46 = 2*(Asym_wood_model$loglik - Asym_wood_model_eq46$loglik) ###order of function should have the better fitting model (higher loglik) listed first
comp.Asym_wood_eq46
pchisq(comp.Asym_wood_eq46, df = 1, lower.tail=FALSE) # It is indeed better.

[1] 4.706438

[1] 0.03004986

# See if dropping 2 makes stuff really good!

### See if better fit can be obtained by making rate 1 equal to zero

In [200]:
trans_mat_z1 = rate.mat.maker(hrm = FALSE, ntraits = 1, nstates = 3, model = "ARD")
trans_mat_z1 = rate.par.drop(trans_mat_z1, drop.par = 1)

In [201]:
Asym_wood_model_z1 = rayDISC(phy = data[[1]],
                        data = endo.matched,
                        charnum = 2, # Specifies the endo+woody trait
                        model = "ARD",
                        node.state = "marginal",
                        rate.mat = trans_mat_z1
                       )

Asym_wood_model_z1

State distribution in data:
States:	1	2	3	
Counts:	52	93	61	
Initializing... 
Finished. Beginning thorough search... 
Finished. Inferring ancestral states using marginal reconstruction. 
Finished. Performing diagnostic tests. 



Fit
     -lnL      AIC     AICc ntax
 -128.141 266.2821 266.5821  206

Rates
           1           2           3
1         NA 0.003131856 0.005109688
2         NA          NA 0.008021773
3 0.00065081 0.018564804          NA

Arrived at a reliable solution 

#### Is the totally asymmetric model still better?

In [202]:
comp.Asym_wood_z1 = 2*(Asym_wood_model$loglik - Asym_wood_model_z1$loglik) ###order of function should have the better fitting model (higher loglik) listed first
comp.Asym_wood_z1
pchisq(comp.Asym_wood_z1, df = 1, lower.tail=FALSE) # It may not be better.

[1] 0.5437167

[1] 0.4608965

### Plot best model

In [ ]:
# . . . 

transition_matrix = rate.mat.maker(hrm=FALSE,ntraits=1,nstates=3,model="ARD") # If the asymmetric model is best, otherwise use model="ER".